In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

batch_size 一次载入多少样本
num_works 多进程，windows大于零，下经常出错
drop_last 如果多余样本是否舍去

In [ ]:
test_data = torchvision.datasets.CIFAR10("./dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)
test_loader = DataLoader(dataset=test_data, batch_size=4, shuffle= True, num_workers=0, drop_last= False)

img,target = test_data[0]
print(img.shape)
print(target)

writer = SummaryWriter("dataLoader")
step = 0
for data in test_loader:
        imgs,targets = data
        print(img.shape)
        print(target)
        writer.add_images("test_data_drop_last",imgs,step)
        step = step + 1
writer.close()

dataset
    getitem():
        return img,target

    dataloader(batch_size = 4)
        return imgs,targets

    img0,target0 = dataset[0]
    img1,targrt1 = dataset[1]
    img2,targrt2 = dataset[2]
    imgs, targets

In [ ]:
from torch import nn


In [ ]:
from torch import nn
import torch

class Tudui(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,input):
        output = input + 1
        return output
    
tudui = Tudui()
x= torch.tensor(1.0)
output = tudui(1)  
print(output)

卷积使用

In [ ]:
import torchvision
from torch.nn import Conv2d
from torch.utils.data import DataLoader
from torch import nn

In [ ]:
dataset = torchvision.datasets.CIFAR10("../data",train=False, transform=torchvision.transforms.ToTensor(),
                                       download=False)
dataloader = DataLoader(dataset,batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding =0)
    
    def forward(self,x):
        x = self.conv1(x)
        return x
tudui = Tudui()

writer = SummaryWriter("../logs")
step = 0

for data in dataloader:
    imgs,target = data
    output = tudui(imgs)
    print(imgs.shape)
    print(output.shape)
    writer.add_images('input',imgs,step)
    
    torch.reshape(output,(-1,3,30,30))
    writer.add_images('output',output,step)
    
    step = step + 1 


        

最大池化的使用
maxpool 下采样
maxuponpool 上采样
kernel_size 取最大值的窗口
stride 步长，默认为1
dilation 空洞卷积
ceil_mode(ceil or floor)向上或者向下取整，当stride过大时，True第二步就会保留不足的。

In [ ]:
import torch
from torch import nn
from torch.nn import MaxPool2d
import torchvision

#取出数据
dataset = torchvision.datasets.CIFAR10("../data", train=False, download=True, 
                                       transform= torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size=64)

# input = torch.tensor(
#     [[3,1,3,2,5],
#     [7,3,5,6,2],
#     [1,3,4,7,9],
#     [4,6,2,1,4],
#     [1,4,6,3,1]], dtype= torch.float32
# )
# input = torch.reshape(input,(-1,1,5,5))
# print(input.shape)

#卷积神经网络基本架构
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size=3, ceil_mode=False)

    def forward(self, input):
        output = self.maxpool1(input)
        return output
    print(output)
    
tudui = Tudui()

writer = SummaryWriter ('../logs_maxpool')
step = 0

#分批次训练
for data in dataloader:
    imgs, target = data
    writer.add_images("input", imgs, step)
    output = tudui(imgs)
    writer.add_images("output", output, step)
    step = step + 1

output = tudui(input)


padding用零或其他常数进行填充
非线性激活RELU，当大于0就取其值，小于0就取0

In [ ]:
import torch
from torch.nn import ReLU
from torch.nn import Sigmoid

input = torch.tensor([[1,2],
                      [-1,3]])
output = torch.reshape(input,(-1,1,2,2))
print(output.shape)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.relu1 = ReLU() #inplace为False时保留原始输入数据
        self.sigmoid1 = Sigmoid()
    
    def forward(self,input):
        output = self.sigmoid1(input)
        return output
    print(output)

tudui = Tudui()
output = tudui(input)




TCN网络的实现

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d,self).__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        """[summary]

        此处指的是剪裁模块，剪裁多出来的padding
        """
        return x[:,:,:-self.chomp_size].contiguous()
    
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """[相当于一个Residual Block]

        Args:
            n_inputs ([int]): [输入通道数]
            n_outputs ([int]): [输出通道数]
            kernel_size ([int]): [卷积核]
            stride ([int]): [步长，一般为1]
            dilation ([int]): [膨胀系数]
            padding ([int]): [填充系数]
            dropout (float, optional): [丢弃比率]. Defaults to 0.2.
        """
        
        super(TemporalBlock, self),__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding,
                                           dilation=dilation))
        #经过conv1,输出的size其实是(Batch_size, input_channel, seq_len+padding)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        
        self.conv2 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding,
                                           dilation=dilation))
        #经过conv1,输出的size其实是(Batch_size, input_channel, seq_len+padding)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1, 
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs !=n_outputs else None
        
        self.relu = nn.ReLU()
        self.init_weights()
        
    def init_weights(self):
        """[参数初始化]
        
        """
        self.conv1.weight.data.normal_(0,0.01)
        self.conv2.weight.data.normal_(0,0.01)
        
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0,0.01)
            
        
    def forward(self, x):
        """[参数]

        Args:
            para x ([int]): [size of Batch, input_channel, seq_len]
        """
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)
    
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout = 0.2):
        """[summary]

        Args:
            num_inputs ([int]): [输入通道数]
            num_channels ([list]): [每层的hidden_channel数，例如[25,25,25,25]表示有4个隐层，每个层有25个神经单元]
            hernel_size (int, optional): [description]. Defaults to 2.
            dropout (float, optional): [description]. Defaults to 0.2.
        """
        super().__init__(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2**i #膨胀系数：1，2，4，8...
            in_channels = num_inputs if i ==0 else num_channels[i-1] #确定每层的输入通道数
            out_channels = num_channels[i] #确定每层时输出通道数
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1)*dilation_size, dropout=dropout)]
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        """
        输入x的结构不同于RNN，一般RNN的size为(Batch, seq_len, channels)或者(seq_len, Batch, channels)，
        这里把seq_len放在channels后面，把所有时间步的数据拼起来，当做Conv1d的输入尺寸，实现卷积跨时间步的操作，
        很巧妙的设计。
        
        :param x: size of (Batch, input_channel, seq_len)
        :return: size of (Batch, output_channel, seq_len)
        """    
        return self.network(x)
tcn = TemporalConvNet(3,[10,10,10,10])
print(tcn)
        
        

线性层

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torch.nn import Linear

dataset = torchvision.datasets.CIFAR10("../data",train = False,transform= torchvision.transforms.ToTensor(),
                                       download = True)

dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.linear = Linear(196608, 10)
    def forward(self, input):
        output = self.linear(input)
        return output

tudui = Tudui()

for data in dataloader:
    imgs, target = data
    print(imgs.shape)
    # output = torch.reshape(imgs,(1,1,1,-1))
    torch.flatten(imgs)
    print(output.shape)
    output = tudui(output)
    print(output.shape)


容器类Sequential 

对CIFAR10进行分类

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.nn import Sequential, Conv2d, Flatten, MaxPool2d, Linear
from torch.utils.tensorboard.writer import SummaryWriter

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(3,32,5,padding=2)
        self.maxpool1 = MaxPool2d(2)
        self.conv2 = Conv2d(32,32,5,padding=2)
        self.maxpool2 = MaxPool2d(2)
        self.conv3 = Conv2d(32,64,5,padding=2)
        self.maxpool3 = MaxPool2d(2)
        self.flatten = Flatten()
        self.linear1 = Linear(1024,64)
        self.linear2 = Linear(64,10)
        
        # self.model = Sequential(
        #     Conv2d(3,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,64,5,padding=2),
        #     MaxPool2d(2),
        #     Flatten(),
        #     Linear(1024,64),
        #     Linear(64,10) 
        # )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x
tudui = Tudui()
print(tudui)

input = torch.ones((64,3,32,32))
output = tudui(input)
print(output)

writer = SummaryWriter("..\logs_seq")
writer.add_graph(tudui, input)
writer.close()
         
        

损失函数与反向传播



In [ ]:
import torch
from torch.nn import L1Loss, CrossEntropyLoss
from torch import nn

inputs= torch.tensor([1,2,3],dtype = torch.float32)
targets = torch.tensor([1,2,5],dtype = torch.float32)

inputs = torch.reshape(inputs,(1,1,1,3))
targets = torch.reshape(targets,(1,1,1,3))

loss = L1Loss(reduction="sum")
result = loss(inputs, targets)

loss_mse = nn.MSELoss()
result_mse = loss_mse(inputs,targets)

print(result)
print(result_mse)

x = torch.tensor([0.1,0.2,0.3])
y = torch.tensor([1])

x = torch.reshape(x,(1,3))
loss_cross = nn.CrossEntropyLoss()
result_cross = loss_cross(x,y)
print(result_cross)

: 

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.nn import Sequential, Conv2d, Flatten, MaxPool2d, Linear
from torch.utils.tensorboard.writer import SummaryWriter


dataset = torchvision.datasets.CIFAR10("../data",train=False,transform=torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset,batch_size=1)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(3,32,5,padding=2)
        self.maxpool1 = MaxPool2d(2)
        self.conv2 = Conv2d(32,32,5,padding=2)
        self.maxpool2 = MaxPool2d(2)
        self.conv3 = Conv2d(32,64,5,padding=2)
        self.maxpool3 = MaxPool2d(2)
        self.flatten = Flatten()
        self.linear1 = Linear(1024,64)
        self.linear2 = Linear(64,10)
        
        # self.model = Sequential(
        #     Conv2d(3,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,64,5,padding=2),
        #     MaxPool2d(2),
        #     Flatten(),
        #     Linear(1024,64),
        #     Linear(64,10) 
        # )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x
tudui = Tudui()
print(tudui)

loss = nn.CrossEntropyLoss()
tudui = Tudui()

for data in dataloader:
    imgs, targets = data
    outputs = tudui(imgs)
    print(outputs)
    print(targets)
    
    result_loss = loss(outputs,targets)
    print(result_loss)


# writer = SummaryWriter("..\logs_seq")
# writer.add_graph(tudui, input)
# writer.close()

: 

optimizer

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.nn import Sequential, Conv2d, Flatten, MaxPool2d, Linear, 
from torch.utils.tensorboard.writer import SummaryWriter


dataset = torchvision.datasets.CIFAR10("../data",train=False,transform=torchvision.transforms.ToTensor(),
                                       download = False)

dataloader = DataLoader(dataset,batch_size=1)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(3,32,5,padding=2)
        self.maxpool1 = MaxPool2d(2)
        self.conv2 = Conv2d(32,32,5,padding=2)
        self.maxpool2 = MaxPool2d(2)
        self.conv3 = Conv2d(32,64,5,padding=2)
        self.maxpool3 = MaxPool2d(2)
        self.flatten = Flatten()
        self.linear1 = Linear(1024,64)
        self.linear2 = Linear(64,10)
        
        # self.model = Sequential(
        #     Conv2d(3,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,64,5,padding=2),
        #     MaxPool2d(2),
        #     Flatten(),
        #     Linear(1024,64),
        #     Linear(64,10) 
        # )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x
tudui = Tudui()
print(tudui)

loss = nn.CrossEntropyLoss()
tudui = Tudui()
#构造优化器
optim = torch.optim.RAdam(tudui.parameters(), lr=0.01)
for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = tudui(imgs)
        print(outputs)
        print(targets)
        
        result_loss = loss(outputs,targets)
        optim.zero_grad()
        result_loss.backward()
        optim.step()
        running_loss = running_loss + result_loss
        
        print(result_loss)


# writer = SummaryWriter("..\logs_seq")
# writer.add_graph(tudui, input)
# writer.close()

: 